<a href="https://colab.research.google.com/github/dookda/cmu_deep_learning/blob/main/YOLO8_sky_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [3]:
!mkdir sky_detect

In [ ]:
!unzip /content/drive/MyDrive/_DATASET/sky.v6i.yolov8.zip -d /content/sky_detect

In [5]:
!rm -r sky_detect/sky.v6i.yolov8/
!rm -r sky_detect/__MACOSX/

install labelme

`pip install labelme

In [ ]:
!pip install ultralytics

`pip install pandas ultralytics numpy matplotlib seaborn pytest-warnings PyYAML random Pillow opencv-python`

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

from ultralytics import YOLO
import cv2
from PIL import Image
import yaml
import random

In [7]:
sns.set(rc={'axes.facecolor': '#ffe4de'}, style='darkgrid')

In [9]:
dataDir = '/content/sky_detect/'

In [ ]:
dataDir = os.path.join(os.getcwd(), 'sky.v6i.yolov8')
print(dataDir)

In [ ]:
trainImagePath = os.path.join(dataDir, 'train', 'images')

imageFiles = [f for f in os.listdir(trainImagePath) if f.endswith('.jpg')]

randomImages = random.sample(imageFiles, 15)

plt.figure(figsize=(10,10))

for i, image_file in enumerate(randomImages):
  image_path = os.path.join(trainImagePath, image_file)
  image = Image.open(image_path)
  plt.subplot(3, 5, i + 1)
  plt.imshow(image)
  plt.axis('off')

plt.suptitle('Random selection of Dataset Images', fontsize=24)

In [20]:
model = YOLO('yolov8n-seg.pt')

In [ ]:
yamlFilePath = os.path.join(dataDir, 'data.yaml')
print(yamlFilePath)

In [ ]:
results = model.train(
    data=yamlFilePath,
    epochs=10,
    imgsz=640,
    batch=32,
    optimizer='auto',
    lr0=0.0001,
    lrf=0.01,
    dropout=0.25,
    device='cpu',
    # device=0,
    seed=42
)

In [15]:
bestModelPath = '/content/runs/segment/train/weights/best.pt'
bestModel = YOLO(bestModelPath)

In [ ]:
validImagePath = os.path.join(dataDir, 'valid', 'images')

imageFiles = [f for f in os.listdir(validImagePath) if f.endswith('.jpg')]

numImages = len(imageFiles)
selectedImage = [imageFiles[i] for i in range(0, numImages, numImages // 9)]

fig, axes = plt.subplots(3, 3, figsize=(10, 10))
fig.suptitle('Validation Set')

for i, ax in enumerate(axes.flatten()):
  imagePath = os.path.join(validImagePath, selectedImage[i])
  results = bestModel.predict(source=imagePath, imgsz=640)
  annotatedImage = results[0].plot()
  annotatedImageRGB = cv2.cvtColor(annotatedImage, cv2.COLOR_BGR2RGB)
  ax.imshow(annotatedImageRGB)
  ax.axis('off')

plt.tight_layout()
plt.show()